In [1]:
import mpg.mpgml.dataset.generator as mpgml_generator

2023-04-05 15:51:08.294985: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [103]:
from importlib import reload
import mpg.wrapper as mpgwrapper
import mpg.mpgml.dataset.generator
from mpg.mpgml.dataset.generator import MPGGeneratedDenseDataset
%load_ext autoreload
%aimport mpg.mpgml.dataset.generator
%aimport mpg.wrapper

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [104]:
%autoreload 2
import tensorflow as tf
import tensorflow_probability as tfp

In [105]:
%autoreload 2
%aimport mpg.mpgml.layers.normalisation 
%aimport mpg.mpgml.layers.augmentation 
%aimport mpg.mpgml.layers.padding 
%aimport mpg.mpgml.dataset.augmentation

n=10
dataset=MPGGeneratedDenseDataset(n,0.5,seed=25,target="winners",generated_input="both",flatten=False,weights_distribution=tfp.distributions.Uniform(-100,100),weight_type="int")
validation_dataset=MPGGeneratedDenseDataset(8,0.5,target="winners",generated_input="both",flatten=False,weights_distribution=tfp.distributions.Uniform(-1,1),weight_type="float")

In [106]:
import mpg.mpgml.dataset.utils as utils
import mpg.mpgml.dataset.augmentation as ds_aug
def assume_starting_position(dataset ,starting_vertex ,starting_turn):
    return dataset.map(lambda x ,y: (x ,y[..., starting_turn ,starting_vertex]))
transformed=assume_starting_position(ds_aug.random_graph_isomorphism(dataset.batch(64).take(1024).cache(),10),0,0).repeat()
validation_dataset=assume_starting_position(validation_dataset,0,0).batch(64).take(12).cache()

In [107]:
validation_dataset=MPGGeneratedDenseDataset(10,0.5,target="winners",generated_input="both",flatten=False,weights_distribution=tfp.distributions.Uniform(-1,1),weight_type="float")
validation_dataset=assume_starting_position(validation_dataset,0,0).batch(64).take(12).cache()


In [108]:
import mpg.mpgml.layers.normalisation as normalisation
import mpg.mpgml.layers.augmentation as augmentation

%aimport mpg.mpgml.layers.normalisation 
%aimport mpg.mpgml.layers.augmentation 
%autoreload 2
layer=normalisation.MPGNormalisationLayer(edges_matrix=1,mask_tensor=0)
A=augmentation.EdgeWeightsNoiseLayer(noise_layer=augmentation.UniformNoise(0.1),edges_matrix=1, mask_tensor=0)
Z=layer(X)
A(Z,training=True),Z

NameError: name 'X' is not defined

In [109]:
for x,y in transformed.take(1):
    print(x)

tf.Tensor(
[[[[  1.          1.          1.        ...   1.          1.
      1.       ]
   [  0.          0.          0.        ...   0.          0.
      1.       ]
   [  0.          1.          1.        ...   1.          1.
      1.       ]
   ...
   [  0.          0.          0.        ...   0.          0.
      0.       ]
   [  1.          1.          0.        ...   1.          1.
      1.       ]
   [  0.          0.          0.        ...   1.          1.
      0.       ]]

  [[ 79.22635     7.849411   -7.637451  ...  16.576195   99.95265
    -98.60854  ]
   [ -0.          0.         -0.        ...  -0.          0.
     -9.235237 ]
   [ -0.        -24.302147   25.138405  ...  77.88005    63.43637
     79.23631  ]
   ...
   [ -0.          0.         -0.        ...   0.         -0.
      0.       ]
   [ 20.46077   -93.29035     0.        ...  93.8745    -73.26996
    -81.58362  ]
   [ -0.          0.          0.        ...  86.60672   -56.62422
     -0.       ]]]


 [[[  0.     

2023-04-08 06:13:22.734215: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [110]:
transformed.take(10).element_spec

(TensorSpec(shape=(None, 2, 10, 10), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.float32, name=None))

In [113]:
import mpg.mpgml.layers.normalisation as mpgml_norm
import mpg.mpgml.layers.augmentation as mpgml_augm
import mpg.mpgml.layers.padding as mpgml_pad
%aimport mpg.mpgml.layers.normalisation 
%aimport mpg.mpgml.layers.augmentation 
%aimport mpg.mpgml.layers.padding 

import tensorflow.keras as keras
model=keras.Sequential([
    mpgml_pad.GraphPaddingLayer(10),
#    mpgml_augm.GraphIsomorphismLayer(10),
    mpgml_norm.MPGNormalisationLayer(edges_matrix=1),
    mpgml_augm.EdgeWeightsNoiseLayer(noise_layer=keras.layers.GaussianNoise(stddev=0.01),edges_matrix=1),
    keras.layers.Flatten(),
    keras.layers.Dense(100,activation="relu"),
    keras.layers.Dense(50,activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(50,activation="relu"),
    keras.layers.Dense(15,activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(15,activation="relu"),
    keras.layers.Dense(1,"sigmoid")
])

In [112]:
model.compile(
    optimizer="Adam",
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"],
)
model.fit(x=transformed,
    epochs=25,
    verbose='auto',
    steps_per_epoch=64,
    shuffle=False,
          validation_data=validation_dataset
         )

Epoch 1/25
40/64 [=================>............] - ETA: 3s - loss: 0.7402 - accuracy: 0.5297

KeyboardInterrupt: 

In [188]:
model.fit(x=transformed,
    epochs=60,
    verbose='auto',
    steps_per_epoch=256,
    shuffle=False,
          validation_data=validation_dataset
         )

Epoch 1/60
256/256 [==============================] - 4s 17ms/step - loss: 0.3095 - accuracy: 0.8660 - val_loss: 1.0890 - val_accuracy: 0.5820
Epoch 2/60
256/256 [==============================] - 2s 7ms/step - loss: 0.2597 - accuracy: 0.8915 - val_loss: 1.1163 - val_accuracy: 0.5846
Epoch 3/60
256/256 [==============================] - 2s 7ms/step - loss: 0.2675 - accuracy: 0.8881 - val_loss: 0.9463 - val_accuracy: 0.6159
Epoch 4/60
256/256 [==============================] - 2s 7ms/step - loss: 0.3160 - accuracy: 0.8631 - val_loss: 0.8769 - val_accuracy: 0.6081
Epoch 5/60
256/256 [==============================] - 2s 7ms/step - loss: 0.2956 - accuracy: 0.8742 - val_loss: 1.1093 - val_accuracy: 0.5951
Epoch 6/60
256/256 [==============================] - 2s 7ms/step - loss: 0.2623 - accuracy: 0.8901 - val_loss: 1.1416 - val_accuracy: 0.5716
Epoch 7/60
256/256 [==============================] - 2s 7ms/step - loss: 0.2563 - accuracy: 0.8914 - val_loss: 1.0005 - val_accuracy: 0.6198
Epoch

In [130]:
x,y=next(iter(transformed))
y

2023-04-04 05:43:34.998569: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


<tf.Tensor: shape=(64, 1), dtype=float32, numpy=
array([[1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.]], dtype=float32)>

In [111]:
graph = mpg.graph.random_graph.gnp_random_mpg(10, 0.5, seed=1, method="fast", loops=True,
                                              distribution="integers", low=0, high=10, endpoint=True)

In [123]:
A = tf.convert_to_tensor(nx.adjacency_matrix(graph, weight=None).todense())

In [2]:
from numba import jit
import networkx as nx
import numpy as np
from typing import Union, TypeVar, Type
import mpg.games.mpg as mpg


1.19 s ± 32.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
import mpg.graph.random_graph as rg
G= rg.gnp_random_mpg(500,0.5,seed=27,distribution="integers",low=-10,high=10,endpoint=True)

In [27]:
%timeit mpg.optimal_strategy_pair(G)

20.2 s ± 170 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
import mpg.wrapper as wrapper
%timeit wrapper.optimal_strategy_pair(G)

664 ms ± 63.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [110]:

def _get_winner(A,W,vertex,player):
    edges = []
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            if A[i,j] == 1:
                edges.append((i,j,W[i,j]))
    C=mpgwrapper.mpgcpp.winners_double_edges_cxx(edges)
    return C[player][vertex]

In [77]:
adjacency_distribution: tfp.distributions.Distribution = tfp.distributions.Bernoulli(probs=0.57)
d=tfp.distributions.Uniform(-10,10)

<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1], dtype=int32)>

In [33]:
bytes(tf.constant([5]))

b'\x05\x00\x00\x00'

In [16]:
from importlib import reload
import mpg.wrapper as mpgwrapper
import time
import mpg.graph.random_graph as rg
times=[]
for k in range(10):
    G=rg.gnp_random_mpg(n=1000,p=0.001,distribution="normal",loc=0,scale=1,loops=True)
    print(f"Processing Graph {k}:",flush=True)
    start=time.time()
    mpgwrapper.mpgcpp.winners_float_edges_cxx([(int(u),int(v),int(G.edges[u,v]["weight"])) for u,v in G.edges])
    end=time.time()
    print(f"Processing Time {end-start}:",flush=True)
    times.append(end-start)

Processing Graph 0:
Processing Time 0.2986409664154053:
Processing Graph 1:
Processing Time 0.2029719352722168:
Processing Graph 2:
Processing Time 0.2950937747955322:
Processing Graph 3:
Processing Time 0.012552738189697266:
Processing Graph 4:
Processing Time 0.24564290046691895:
Processing Graph 5:
Processing Time 0.01729297637939453:
Processing Graph 6:
Processing Time 0.0193173885345459:
Processing Graph 7:
Processing Time 0.3035721778869629:
Processing Graph 8:
Processing Time 0.07518315315246582:
Processing Graph 9:
Processing Time 0.2042698860168457:


In [31]:
from importlib import reload
import mpg.wrapper as mpgwrapper
import time
import mpg.graph.random_graph as rg
import numba
import numpy as np
import tensorflow as tf
import mpg.graph.random_graph
import networkx as nx
import itertools
import tensorflow_probability as tfp
import mpg.wrapper as mpgwrapper


def _adj_matrix_generator(n,p):
    
    A=np.zeros([n,n],dtype=np.uint8)
    for k in range(n):
        A[k,:]=np.random.binomial(1,p,n)
        while A[k,:].sum() == 0:
            A[k,:]=np.random.binomial(1,p,n)
    return A

def _generate_dense_instances(n, p, seeder, cardinality: int, target: bool, weight_matrix: bool, flatten: bool,
                              weight_distribution: tfp.distributions.Distribution, weight_type):
    adjacency_distribution: tfp.distributions.Distribution = tfp.distributions.Bernoulli(probs=p,dtype=tf.bool)
    turn_distribution: tfp.distributions.Distribution = tfp.distributions.Bernoulli(probs=0.5)
    # discrete=tfp.distributions.DiscreteUniform(low=0,high=10)
    shape = (n, n) if not flatten else (n * n,)
    W = weight_distribution.sample(shape, seed=seeder())
    dtype=weight_distribution.dtype
    A=tf.numpy_function(_adj_matrix_generator,inp=[n,p],Tout=tf.uint8,stateful=False)
    if flatten:
        A=tf.reshape(A,shape=(-1,))
    A = tf.cast(A,dtype=dtype)
    W = tf.multiply(A, W)
    vertex = tf.random.uniform((1,), 0, n, dtype=tf.int32, seed=seeder())
    player = turn_distribution.sample((1,), seed=seeder())
    if flatten:
        if weight_matrix:
            output = tf.concat(cast_all(dtype,A, W, vertex, player), axis=0)
        else:
            output = tf.concat(cast_all(dtype,A, vertex, player), axis=0)
        if target:
            if weight_type == tf.int32 or weight_type == tf.int64:
                target_value = tf.py_function(
                    lambda output: np.array(mpgwrapper.mpgcpp.winners_tensorflow_int_matrix_flattened_cxx(output.numpy().astype(np.int32).tolist())),
                    inp=[output], Tout=tf.int32)
            else:
                target_value = tf.py_function(
                    lambda output: np.array(mpgwrapper.mpgcpp.winners_tensorflow_float_matrix_flattened_cxx(output.numpy().astype(np.float32).tolist())),
                    inp=[output], Tout=tf.float32)
            target_value = tf.reshape(tf.ensure_shape(target_value, ()), shape=(1,))
            return (tf.cast(output,dtype=tf.float32), tf.cast(target_value, dtype=tf.float32))
        return output
    else:
        if weight_matrix:
            output = tf.cast(tf.stack([A, W], axis=0), dtype=tf.float32)
        else:
            output = tf.cast(A, dtype=tf.float32)
        if target:
            return (output, tf.constant([vertex, player]), 1)
        return (output, tf.constant([vertex, player]))


def cast_all(dtype,*args):
    return tuple(tf.cast(arg, dtype) for arg in args)
times=[]
with tf.device("/device:gpu:0") as device:
    for k in range(10):
        print(f"Processing Graph {k}:",flush=True)
        seeder=tfp.util.SeedStream(k,"k")
        start=time.time()
        _generate_dense_instances(n=200,p=1,seeder=seeder,cardinality=1,target=False,weight_matrix=True,flatten=True,weight_distribution=tfp.distributions.Uniform(-30,30),weight_type="int")
        end=time.time()
        print(f"Processing Time {end-start}:",flush=True)
        times.append(end-start)

Processing Graph 0:
Processing Time 0.04545021057128906:
Processing Graph 1:
Processing Time 0.014969825744628906:
Processing Graph 2:
Processing Time 0.01526021957397461:
Processing Graph 3:
Processing Time 0.013864278793334961:
Processing Graph 4:
Processing Time 0.014152050018310547:
Processing Graph 5:
Processing Time 0.012566089630126953:
Processing Graph 6:
Processing Time 0.012970209121704102:
Processing Graph 7:
Processing Time 0.014101028442382812:
Processing Graph 8:
Processing Time 0.012717485427856445:
Processing Graph 9:
Processing Time 0.013520479202270508:


In [2]:

def _adj_matrix_generator(n,p):
    A=np.zeros([n,n],dtype=np.uint8)
    for k in range(n):
        A[k,:]=np.random.binomial(1,p,n)
        while A[k,:].sum() == 0:
            A[k,:]=np.random.binomial(1,p,n)
    return A
_adj_matrix_generator(100,0.5)

array([[0, 1, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 1, 0, 1],
       ...,
       [1, 1, 0, ..., 1, 0, 0],
       [0, 1, 1, ..., 0, 1, 1],
       [0, 1, 0, ..., 1, 0, 0]], dtype=uint8)

In [8]:
import time
n=100
seeder=tfp.util.SeedStream(4,"g")
X=generate_dense_gnp_instance(n=n,p=0.1,seeder=seeder,cardinality=1,
                          target="none",weight_matrix=True,flatten=True,weight_distribution=tfp.distributions.Uniform(-100,100),weight_type="int")
L=[]
Z=X.numpy()
for i in range(n):
    for j in range(n):
        if Z[i+n*j] == 1:
            L.append((i,j,int(Z[i+n*j+n*n])))
a=time.time()
Q=mpgwrapper.mpgcpp.targets_tensorflow_int_cxx(X.numpy().reshape((2,n,n)).astype(np.int32).tolist(),"dense","all",False)
b=time.time()
print(b-a)

0.0311582088470459


<tf.Tensor: shape=(2, 2, 100), dtype=int32, numpy=
array([[[88, 85, 80, 77, 91, 95, 88, 96, 98, 93, 98, 96, 97, 99, 93, 96,
         95, 97, 89, 98, 95, 99, 95, 99, 92, 93, 99, 86, 97, 94, 96, 98,
         94, 95, 86, 99, 91, 88, 95, 85, 81, 93, 93, 81, 91, 96, 69, 92,
         99, 95, 95, 84, 90, 95, 87, 98, 65, 98, 94, 71, 98, 90, 96, 89,
         95, 97, 97, 84, 90, 85, 93, 99, 97, 97, 91, 90, 98, 88, 99, 91,
         90, 78, 97, 81, 99, 93, 78, 96, 95, 94, 92, 98, 96, 95, 86, 97,
         95, 96, 97, 85],
        [88, 81, 15, 12,  5, 38, 88, 84, 81, 29, 19, 34, 88,  2, 29, 43,
         71, 95,  2, 16, 81, 84, 95, 97, 61, 11, 76,  6, 38, 55, 96, 88,
         94, 46, 84, 14, 42, 88, 95, 49, 68, 68, 50, 81, 33, 96, 69, 29,
         99, 95, 46, 84, 50,  5, 53, 86, 64, 34, 42, 55, 76, 49, 96, 19,
         34, 97,  4, 38,  4, 42,  5, 87,  3,  2, 71, 23, 55, 78, 84, 84,
         25, 31, 80, 57, 91,  4, 76, 17, 95, 84, 78, 80, 23, 95, 63, 68,
         95, 10, 84, 61]],

       [[ 1,  1,  1

In [13]:
seeder=tfp.util.SeedStream(45,"a")
n=10
X=_generate_dense_instances(n=n,p=0.01,seeder=seeder,cardinality=1,
                          target="none",weight_matrix=True,flatten=True,weight_distribution=tfp.distributions.Uniform(-30,30),weight_type="int")

In [30]:
tf.py_function(lambda X:np.array([[1,2],[3,4]]),inp=[X],Tout=tf.TensorSpec([5,10]))

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>

In [11]:
Z=X.numpy()
Z=Z[n*n:2*n*n].reshape((n,n))
Z=Z.astype(np.int32)
Z
A=X.numpy()[:n*n].reshape((n,n))

NameError: name 'X' is not defined

In [52]:
import networkx as nx
import mpg.games.mpg as mpg
G=nx.from_numpy_array(A,create_using=mpg.MeanPayoffGraph)
for e in G.edges:
    u,v=e
    G.edges[u,v]["weight"]=Z[u,v]

In [ ]:
mpgwrapper.mean_payoffs(G)

In [10]:
tf.constant(((1,1),(2,2)))

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1, 1],
       [2, 2]], dtype=int32)>

ValueError: Exception encountered when calling layer 'laplacian_matrix_layer_4' (type LaplacianMatrixLayer).

Attempt to convert a value (<TakeDataset element_spec=(TensorSpec(shape=(None, 200), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>) with an unsupported type (<class 'tensorflow.python.data.ops.dataset_ops.TakeDataset'>) to a Tensor.

Call arguments received by layer 'laplacian_matrix_layer_4' (type LaplacianMatrixLayer):
  • inputs=<TakeDataset element_spec=(TensorSpec(shape=(None, 200), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>
  • kwargs={'training': 'None'}

In [74]:
z=tf.constant([[1,2],[3,4]])
M=tf.ragged.boolean_mask(z,[[True,True],[True,False]])

In [125]:
tf.math.reduce_std(M,axis=-1)

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.5, 0. ])>

In [126]:
M

<tf.RaggedTensor [[1, 2], [3]]>

In [161]:
tf.constant([1,1])==1

<tf.Tensor: shape=(2,), dtype=bool, numpy=array([ True,  True])>

In [162]:
p=[1,0]
tf.gather(tf.gather(tf.stack([z,z,z,z]),p,axis=-2),p,axis=-1)

<tf.Tensor: shape=(4, 2, 2), dtype=int32, numpy=
array([[[4, 3],
        [2, 1]],

       [[4, 3],
        [2, 1]],

       [[4, 3],
        [2, 1]],

       [[4, 3],
        [2, 1]]], dtype=int32)>

In [86]:
z

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4]], dtype=int32)>

In [93]:
tf.random_index_shuffle(
    [0,1,2], [55,1,2], 3, rounds=4, name=None
)


<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 0], dtype=int32)>

In [34]:
import tensorflow as tf
class MPGFileReader(tf.data.Dataset):
    def __init__(self,k):
        self.k=k
        super().__init__(k)
    def _inputs(self):
        return []
        pass

    def element_spec(self):
        return tf.TypeSpec(shape=())
        pass


In [35]:
x=MPGFileReader(5)

In [36]:
for s in x:
    print(s)

AttributeError: 'function' object has no attribute '_flat_tensor_specs'

In [118]:
%autoreload 2
%aimport mpg.mpgml.layers.normalisation 
%aimport mpg.mpgml.layers.augmentation 
%aimport mpg.mpgml.layers.padding 
%aimport mpg.mpgml.dataset.augmentation

n=100
dataset=mpg.mpgml.dataset.generator.MPGSparseGeneratedDataset(n,0.1,seed=25,target="none",generated_input="weight",flatten=False,weights_distribution=tfp.distributions.Uniform(-100,100),weight_type="int")

In [119]:
for x in dataset.batch(64).take(1):
    print(x)

SparseTensor(indices=tf.Tensor(
[[ 0  0 37]
 [ 0  0  8]
 [ 0  0 58]
 ...
 [63 99  6]
 [63 99  0]
 [63 99 58]], shape=(64237, 3), dtype=int64), values=tf.Tensor([ 96.56076   42.62047   54.516464 ... -16.267868 -96.6578    67.36159 ], shape=(64237,), dtype=float32), dense_shape=tf.Tensor([ 64 100 100], shape=(3,), dtype=int64))


In [115]:
model.compile(
    optimizer="Adam",
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"],
)
model.fit(x=transformed,
    epochs=25,
    verbose='auto',
    steps_per_epoch=64,
    shuffle=False,
          validation_data=validation_dataset
         )

Epoch 1/25
50/64 [======================>.......] - ETA: 2s - loss: 0.7219 - accuracy: 0.5437

KeyboardInterrupt: 